# Honeyword Encryption fdasf

**Plan projektu:**

1. historia dobrze wykorzystanej przynęty
2. wstępne informacje o temacie
3. implementacja w obszarze kart płatniczych

### 1. Operacja Micemeat

30 kwietnia 1943 roku na hiszpańskim wybrzeżu zostało znalezione ciało majora Royal Marines przez tamtejszego rybaka. Pomimo neutralności Hiszpanii w okolicy znajdował się niemiecki posterunek, którego agent zainteresował się wrogimi zwłokami. Zmarły posiadał przy sobie dokumenty, które okazały się dla niezwykle cenne dla Abwehry - zawierały informacje o planowanej dużej inwazji aliantów na **Grecję**. Niemcy przypuszczali wystąpienie inwazji, ale brakowało im informacji o miejscu. Przechowony list skutkwał rozkazem Führera, który przemieścił trzy pancerne dywizje na wybrzeża Grecji.

<details>
<br>
Inwazja została przeprowadzona na Sycylię. Znaleziony major nigdy nie istniał, jego tożsamość została stworzona na potrzeby operacji przez brytyjski wywiad. Oczywiście celem było wprowadzenie wroga w błąd.
<br>
<br>
Podczas fabrykacji żołnierza dołożono wszelkich starań dla zagwarantowania autentyczności - przy jego ciele znajdowały się też zdjęcie abstrakcyjnej narzeczonej, listy niepocieszonego ojca, list wychwalający majora jako eksperta w dziedzinie operacji desantowych, prawdziwe bilety z kina, ale również nota z banku o przekroczeniu stanu konta, niezapłacony rachunek i przepustka, która straciła ważność.
<br>
Ponadto na łamach dziennika The Times opublikowano nazwisko majora na liście ofiar brytyjskich. Do władz Hiszpanii wysłano ciąg pilnych depesz, w których brytyjczycy upominali się o zwrot dokumentów (z należytym zachowaniem dyskrecji). Dzięki odpowiednim przygotowaniu dokumentów, po ich odesłaniu brytyjscy specjaliści byli w stanie potwierdzić, że zostały one wcześniej otwarte.
<br>
<br>
Powyższe działania pozwoliły na względnie niewielki opór przeciwnika podczas inwazji. Co więcej miały też wpływ na późniejsze operacje - Niemcy przestali wierzyć znajdowanym dokumentom.
</details>